В своей основе декораторы Python позволяют расширять и изменять поведение вызываемых объектов (функций, методов и классов) без необратимой модификации самих вызываемых объектов.

Любая достаточно универсальная функциональность, которую можно прикрепить к существующему классу или поведению функции, является отличным кандидатом для декорирования. Сюда входят:

- ведение протокола операций (журналирование);
- обеспечение контроля за доступом и аутентификацией;
- функции инструментального оформления и хронометража;
- ограничение частоты вызова API;
- кэширование и др.

Предположим, в вашей программе составления отчетности есть 30 функций с бизнес-логикой. Одним дождливым утром понедельника ваш босс подходит к вашему столу и заявляет: 

«Доброго понедельника! Помните ту отчетность по TPS? Мне нужно, чтобы вы в каждый шаг генератора отчетов добавили ведение протокола входных и выходных операций. Компании XYZ это нужно для аудиторских целей. Да, и еще. Я им сказал, что к среде мы сможем все отправить». 

Без декораторов следующие три дня вам пришлось бы провести в попытках модифицировать каждую из этих 30 функций, приводя их в полный беспорядок ручными вызовами операции журналирования. Чудесно, не правда ли? 
А если вы знаете свои декораторы, вы спокойно улыбнетесь своему боссу и скажете: «Не беспокойся, Джим. Я сделаю это сегодня к 14:00».

Декораторы позволяют определять конструктивные блоки многократного использования, которые могут изменять или расширять поведение других функций. И они позволяют это делать без необратимых изменений самой
обернутой функции. Поведение функции изменяется, только когда оно декорировано.

Декоратор — это вызываемый объект, который на входе принимает один вызываемый объект, а на выходе возвращает другой вызываемый объект. 
Приведенная ниже функция имеет это свойство и может считаться самым простым декоратором, который вы могли когда-либо написать:

In [1]:
def null_decorator(func):
    return func

Как вы видите, **null_decorator** является вызываемым объектом (это функция). На входе он принимает еще один вызываемый объект и на выходе возвращает тот же самый вызываемый объект без его изменения.

Давайте его применим, чтобы декорировать (или обернуть) еще одну функцию:

In [2]:
def greet():
    return 'Привет!'

In [3]:
greet = null_decorator(greet)

In [4]:
greet()

'Привет!'

In [5]:
greet

<function __main__.greet()>

In [6]:
null_decorator(greet)

<function __main__.greet()>

В этом примере я определил функцию **greet** и сразу же ее декорировал, пропустив через функцию **null_decorator**. Понимаю, пока это все выглядит бесполезным. Я ведь о том, что мы намеренно  спроектировали пустой декоратор бесполезным, верно? Но через мгновение этот пример разъяснит, как работает специальный синтаксис Python, предназначенный для декораторов.

Вместо того чтобы явным образом вызывать **null_decorator** с функцией **greet** и затем по-новому присваивать его переменной, удобнее воспользоваться синтаксисом Python **@** для декорирования функции:

In [7]:
@null_decorator
def greet():
    return 'Привет!'

In [8]:
greet()

'Привет!'

Размещение строки **@null_decorator** перед определением функции аналогично тому, что функция сначала определяется и затем уже прогоняется через декоратор. Синтаксис **@** является всего лишь *синтаксическим сахаром (syntactic sugar)* и краткой формой для этого широко применяемого шаблона.

Обратите внимание: синтаксис **@** декорирует функцию непосредственно во время ее определения. При этом становится трудно получить доступ к недекорированному оригиналу без хрупких хакерских фокусов. По этой
причине вы можете решить вручную декорировать некоторые функции для сохранения способности вызвать и недекорированную функцию.

## Декораторы могут менять поведение

Теперь, когда вы чуть ближе познакомились с синтаксисом декораторов, давайте напишем еще один декоратор, который действительно что-то делает и изменяет поведение декорированной функции.

Вот чуть более сложный декоратор, который преобразовывает результат декорированной функции в буквы верхнего регистра:

In [9]:
def uppercase(func):
    def wrapper():
        original_result = func()
        modified_result = original_result.upper()
        return modified_result
    return wrapper

Вместо того чтобы просто возвратить входную функцию, как это делал пустой декоратор, декоратор **uppercase** на лету определяет новую функцию *(замыкание)* и использует ее в качестве обертки входной функции, чтобы изменить ее поведение во время вызова.

Замыкание **wrapper** имеет доступ к недекорированной входной функции, и оно свободно может выполнить дополнительный программный код до и после ее вызова. (Технически замыканию вообще не нужно вызывать
входную функцию.) 

Заметьте, что вплоть до настоящего момента декорированная функция ни разу не была исполнена. На самом деле, в вызове входной функции на данном этапе нет никакого смысла — потребность в том, чтобы декоратор
был в состоянии изменить поведение своей входной функции, возникнет, только когда он наконец будет вызван.

In [10]:
@uppercase
def greet():
    return 'Привет!' 

In [11]:
greet()

'ПРИВЕТ!'

В отличие от **null_decorator**, декоратор **uppercase** при декорировании функции возвращает другой объект-функцию:

In [12]:
uppercase(greet) 

<function __main__.uppercase.<locals>.wrapper()>

И как вы видели чуть раньше, ему это нужно, чтобы изменить поведение декорированной функции, когда он в итоге будет вызван. Декоратор **uppercase** сам является функцией. И единственный способ повлиять на
«будущее поведение» входной функции, которую он декорирует, состоит в том, чтобы подменить (или обернуть) входную функцию замыканием.

Вот почему декоратор **uppercase** определяет и возвращает еще одну функцию (замыкание), которая затем может быть вызвана в дальнейшем, выполняет оригинальную входную функцию и модифицирует ее результат.

Декораторы изменяют поведение вызываемого объекта посредством обертки-замыкания, в результате чего вам не приходится необратимо модифицировать оригинал. Оригинальный вызываемый объект не изменяется необратимо — его поведение меняется, только когда он декорирован.

Это позволяет прикреплять к существующим функциям и классам конструктивные блоки многократного использования, в частности функционал журналирования и другое инструментальное оформление. Этот
факт делает декораторы настолько мощным функциональным средством языка, что они нередко используются в стандартной библиотеке Python и в сторонних пакетах.

## Применение многочисленных декораторов к функции

Возможно, вас не удивит, что к функции можно применять более одного декоратора. В этом случае их эффекты накапливаются, и именно этот факт делает декораторы столь полезными в качестве структурных блоков многократного использования.

Приведем пример. Представленные ниже два декоратора обертывают выходную строку декорированной функции в HTML-теги. Глядя на то, как теги вложены, вы видите, в каком порядке Python применяет многочисленные декораторы:

In [13]:
def strong(func):
    def wrapper():
        return '<strong>' + func() + '</strong>'
    return wrapper

def emphasis(func):
    def wrapper():
        return '<em>' + func() + '</em>'
    return wrapper

Теперь давайте возьмем эти два декоратора и одновременно применим их к нашей функции greet. Для этого вы можете использовать обычный синтаксис **@** и просто «уложить» многочисленные декораторы вертикально
поверх одной-единственной функции:

In [14]:
@strong
@emphasis
def greet():
    return 'Привет!'

In [15]:
greet()

'<strong><em>Привет!</em></strong>'

Этот результат ясно показывает, в каком порядке декораторы были применены: снизу вверх. Сначала входная функция была обернута декоратором **@emphasis**, и затем результирующая (декорированная) функция снова
была обернута декоратором **@strong**.

Если разложить приведенный выше пример и избавиться от синтаксиса **@**, который применяют декораторы, то цепочка вызовов функций-декораторов выглядит так:     

In [16]:
decorated_greet = strong(emphasis(greet))

In [17]:
decorated_greet()

'<strong><em><strong><em>Привет!</em></strong></em></strong>'

## Декорирование функций, принимающих аргументы

Все примеры пока что декорировали только простую нульарную функцию **greet**, которая вообще не принимала никаких аргументов. 
Вплоть до этого момента декораторам, которые вы здесь видели, не было дела до переадресации аргументов во входную функцию.
Если применить один из этих декораторов к функции, которая принимает аргументы, то она не заработает правильно. Тогда как декорировать функцию, которая принимает произвольные аргументы?

### Вспомним про \*args и \*\*kwargs

In [18]:
def foo(required, *args, **kwargs):
    print(required)
    if args:
        print(args)
    if kwargs:
        print(kwargs)

In [19]:
foo(1, 2, 3, 4, key1=5, key2=6)

1
(2, 3, 4)
{'key1': 5, 'key2': 6}


Приведенная выше функция требует по крайней мере одного аргумента под названием «required», то есть обязательный, но она также может принимать дополнительные позиционные и именованные аргументы.

Если мы вызовем функцию с дополнительными аргументами, то **args** соберет дополнительные позиционные аргументы в кортеж, потому что имя параметра имеет префикс \*.

Аналогичным образом, **kwargs** соберет дополнительные именованные аргументы в словарь, потому что имя параметра имеет префикс \*\*.

Сразу хочу прояснить. Название параметров **args** и **kwargs** принято по договоренности, как согласованное правило именования. Приведенный выше пример будет работать точно так же, если вы назовете их \*parms и \*\*argv. Фактическим синтаксисом является, соответственно, просто звездочка (\*) или двойная звездочка (\*\*).

### Вспомним про распаковку аргументов

Существует возможность передавать необязательные или именованные параметры из одной функции в другую. Это можно делать при помощи операторов распаковки аргументов \* и \*\* во время вызова функции, в которую вы хотите переадресовать аргументы.

Это также дает вам возможность модифицировать аргументы перед тем, как вы передадите их дальше. Вот пример:

In [20]:
def bar(a, b, c, name='Ivan', color='red'):
    print(a, b, c, name, color)

In [21]:
def foo(x, *args, **kwargs):
    kwargs['name'] = 'Alice'
    new_args = args + (3, )
    bar(x, *new_args, **kwargs)

In [22]:
foo(1, 2, color='green')

1 2 3 Alice green


Вот где на помощь приходят функциональные средства языка Python \*args и \*\*kwargs для работы с неизвестными количествами аргументов. 

Ниже приведен декоратор **proxy**, в котором задействуется их преимущество:

In [23]:
def proxy(func):
    def wrapper(*args, **kwargs):
        return func(*args, **kwargs)
    return wrapper

С этим декоратором происходят две вещи, заслуживающие внимания:
- В определении замыкания **wrapper** он использует операторы \* и \*\*, чтобы собрать все позиционные и именованные аргументы, и помещает их в переменные (**args** и **kwargs**).
- Замыкание **wrapper** затем переадресует собранные аргументы в оригинальную входную функцию, используя операторы «распаковки аргументов» \* и \*\*.

Давайте расширим прием, сформулированный декоратором **proxy**, в более полезный практический пример. 
Ниже приведен декоратор **trace**, который регистрирует аргументы функции и итоговые результаты, полученные во время исполнения:

In [24]:
def trace(func):
    def wrapper(*args, **kwargs):
        print(f'ТРАССИРОВКА: вызвана {func.__name__}() с {args}, {kwargs}')
        original_result = func(*args, **kwargs)
        print(f'ТРАССИРОВКА: {func.__name__}() вернула {original_result}')
        return original_result
    return wrapper

При декорировании функции с использованием декоратора **trace** и последующем ее вызове, будут выведены переданные в декорированную функцию аргументы и возвращаемое ею значение. 
Этот пример по-прежнему остается несколько «игрушечным» — но в случае крайней необходимости он становится отличным средством отладки:

In [25]:
@trace 
def say(name, line):
    return f'{name}: {line}'

In [26]:
say('Джейн', 'Привет, Мир')

ТРАССИРОВКА: вызвана say() с ('Джейн', 'Привет, Мир'), {}
ТРАССИРОВКА: say() вернула Джейн: Привет, Мир


'Джейн: Привет, Мир'

## Как писать «отлаживаемые» декораторы

При использовании декоратора вы на самом деле только подменяете одну функцию другой. Оборотной стороной этого процесса является то, что он «скрывает» некоторые метаданные, закрепленные за оригинальной (недекорированной) функцией.

Например, оригинальное имя функции, ее строка документации docstring и список параметров скрыты замыканием-оберткой:

In [27]:
def greet():
    """Вернуть дружеское приветствие."""
    return 'Привет!'

In [28]:
decorated_greet = uppercase(greet)

При попытке получить доступ к каким-либо из этих метаданных функции вместо них вы увидите метаданные замыкания-обертки:

In [29]:
greet.__name__

'greet'

In [30]:
greet.__doc__

'Вернуть дружеское приветствие.'

In [31]:
decorated_greet.__name__

'wrapper'

In [32]:
decorated_greet.__doc__

Это делает отладку и работу с интерпретатором Python неуклюжей и трудоемкой. 
К счастью, существует быстрое решение этой проблемы: декоратор **functools.wraps**, включенный в стандартную библиотеку Python.

Декоратор **functools.wraps** можно использовать в своих собственных декораторах для того, чтобы копировать потерянные метаданные из недекорированной функции в замыкание декоратора. Вот пример:

In [33]:
import functools
def uppercase(func):
    @functools.wraps(func)
    def wrapper():
        return func().upper()
    return wrapper

Применение декоратора **functools.wraps** к замыканию-обертке, возвращаемому декоратором, переносит в него строку документации и другие метаданные входной функции:

In [34]:
@uppercase 
def greet():
    """Вернуть дружеское приветствие."""
    return 'Привет!'

In [35]:
greet.__name__

'greet'

In [36]:
greet.__doc__

'Вернуть дружеское приветствие.'

В качестве оптимального практического приема я порекомендовал бы использовать декоратор **functools.wraps** во всех декораторах, которые вы пишете сами. Это не займет много времени и уменьшит головную боль вам (и другим) в будущем при отладке.

## Измерение времени работы программы

In [37]:
def benchmark(func):
    import time    
    def wrapper():
        start = time.time()
        func()
        end = time.time()
        print('[*] Время выполнения: {} секунд.'.format(end-start))
    return wrapper

@benchmark
def fetch_webpage():
    import requests
    webpage = requests.get('https://google.com')

fetch_webpage()

[*] Время выполнения: 0.41301465034484863 секунд.


In [38]:
def benchmark(func):
    import time    
    def wrapper(*args, **kwargs):
        start = time.time()
        return_value = func(*args, **kwargs)
        end = time.time()
        print('[*] Время выполнения: {} секунд.'.format(end-start))
        return return_value
    return wrapper

@benchmark
def fetch_webpage(url):
    import requests
    webpage = requests.get(url)
    return webpage.text[:100]

webpage = fetch_webpage('https://google.com')
print(webpage)

[*] Время выполнения: 0.2400069236755371 секунд.
<!doctype html><html itemscope="" itemtype="http://schema.org/WebPage" lang="ru"><head><meta content


In [39]:
def benchmark(iters):
    def actual_decorator(func):
        import time        
        def wrapper(*args, **kwargs):
            total = 0
            for i in range(iters):
                start = time.time()
                return_value = func(*args, **kwargs)
                end = time.time()
                total = total + (end-start)
            print('[*] Среднее время выполнения: {} секунд.'.format(total/iters))
            return return_value
        return wrapper
    return actual_decorator


@benchmark(iters=10)
def fetch_webpage(url):
    import requests
    webpage = requests.get(url)
    return webpage.text[:100]

webpage = fetch_webpage('https://google.com')
print(webpage)

[*] Среднее время выполнения: 0.2605084180831909 секунд.
<!doctype html><html itemscope="" itemtype="http://schema.org/WebPage" lang="ru"><head><meta content


In [1]:
def fib1(n: int) -> int:
    return fib1(n - 1) + fib1(n - 2)

print(fib1(5))

RecursionError: maximum recursion depth exceeded

In [2]:
def fib2(n: int) -> int:
    if n < 2:  # base case
        return n
    return fib2(n - 2) + fib2(n - 1)  # recursive case

print(fib2(5))
print(fib2(10))

5
55


In [4]:
from typing import Dict
memo: Dict[int, int] = {0: 0, 1: 1}  # our base cases

def fib3(n: int) -> int:
    if n not in memo:
        memo[n] = fib3(n - 1) + fib3(n - 2)  # memoization
    return memo[n]

print(fib3(5))
print(fib3(50))

5
12586269025


In [5]:
from functools import lru_cache

@lru_cache(maxsize=None)
def fib4(n: int) -> int:  # same definition as fib2()
    if n < 2:  # base case
        return n
    return fib4(n - 2) + fib4(n - 1)  # recursive case

print(fib4(5))
print(fib4(50))

5
12586269025


In [6]:
def fib5(n: int) -> int:
    if n == 0: return n  # special case
    last: int = 0  # initially set to fib(0)
    next: int = 1  # initially set to fib(1)
    for _ in range(1, n):
        last, next = next, last + next
    return next

print(fib5(2))
print(fib5(50))

1
12586269025


In [7]:
from typing import Generator

def fib6(n: int) -> Generator[int, None, None]:
    yield 0  # special case
    if n > 0: yield 1  # special case
    last: int = 0  # initially set to fib(0)
    next: int = 1  # initially set to fib(1)
    for _ in range(1, n):
        last, next = next, last + next
        yield next  # main generation step

for i in fib6(50):
    print(i)

0
1
1
2
3
5
8
13
21
34
55
89
144
233
377
610
987
1597
2584
4181
6765
10946
17711
28657
46368
75025
121393
196418
317811
514229
832040
1346269
2178309
3524578
5702887
9227465
14930352
24157817
39088169
63245986
102334155
165580141
267914296
433494437
701408733
1134903170
1836311903
2971215073
4807526976
7778742049
12586269025


## Задание 1

Напишите декоратор **log**

```python
>>> @log
... def function(*args):
...... return 3 + len(args)

>>> function(4, 4, 4)
вы вызвали функцию function(4, 4, 4)
она вернула значение 6
6
```

## Задание 2

Напишите декоратор **html**, который в результате вызова **function('hi')** записывает в файл с именем "my_file.html" следующий текст:

2019/11/27 10:55
```html
<html>
<strong>'hi'</strong>
</html>
```

*Подсказка:* время можно взять из time.strftime()

Пример вызова декоратора:

```python
>>> @html
... def function(s='hello'):
...... return s

>>> function('hi')
```

## Задание 3

Напишите еще одну функцию, которая вычисляет n-й элемент последовательности Фибоначчи, используя метод вашей собственной разработки. Напишите модульные тесты, которые оценивали бы правильной этой функции и ее производительность, по сравнению с другими версиями.